In [1]:
%%javascript
$.getScript("http://120.78.95.32/j_n_contents.js")

<IPython.core.display.Javascript object>

<div id="toc">
<div>

# 6.1 语言模型

# 6.2 循环神经网络

In [2]:
import torch
import torch.nn as nn

In [31]:
x,w_xh=torch.randn(3,1),torch.randn(1,4)
h,w_hh=torch.randn(3,4),torch.randn(4,4)
torch.matmul(x,w_xh)+torch.matmul(h,w_hh)

tensor([[-2.6263,  1.2154,  0.2870,  0.3547],
        [-2.5293,  2.8904, -2.9870, -0.3797],
        [ 1.9435, -0.9750, -0.8919,  0.7421]])

In [32]:
torch.matmul(torch.cat((x,h),dim=1),torch.cat((w_xh,w_hh),dim=0))

tensor([[-2.6263,  1.2154,  0.2870,  0.3547],
        [-2.5293,  2.8904, -2.9870, -0.3797],
        [ 1.9435, -0.9750, -0.8919,  0.7421]])

# 6.3 语言模型数据集(周杰伦专辑歌词)

In [50]:
import torch
import random
import zipfile

with zipfile.ZipFile("../../data/jaychou_lyrics.txt.zip") as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars=f.read().decode('utf-8')
corpus_chars[:40]

'想要有直升机\n想要和你飞到宇宙去\n想要和你融化在一起\n融化在宇宙里\n我每天每天每'

In [51]:
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
corpus_chars = corpus_chars[0:10000]

In [52]:
idx_to_char=list(set(corpus_chars))
char_to_idx=dict([(char,i) for i,char in enumerate(idx_to_char)])
vocab_size=len(char_to_idx)
vocab_size

1027

In [53]:
corpus_indices=[char_to_idx[char] for char in corpus_chars]
sample=corpus_indices[:20]
print("".join([idx_to_char[idx] for idx in sample]))
print(sample)

想要有直升机 想要和你飞到宇宙去 想要和
[802, 492, 828, 567, 673, 789, 920, 802, 492, 957, 944, 86, 739, 704, 801, 97, 920, 802, 492, 957]


In [54]:
def data_iter_random(corpus_indices,batch_size,num_steps,device=None):
    num_examples=(len(corpus_indices)-1)//num_steps
    epoch_size=num_examples//batch_size
    example_indices=list(range(num_examples))
    random.shuffle(example_indices)
    
    def _data(pos):
        return corpus_indices[pos:pos+num_steps]
    if device is None:
        device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
       
    for i in range(epoch_size):
        i=i*batch_size
        batch_indices=example_indices[i:i+batch_size]
        x=[_data(j*num_steps) for j in batch_indices]
        y=[_data(j*num_steps+1) for j in batch_indices]
        
        yield torch.tensor(x,dtype=torch.float32,device=device),torch.tensor(y,dtype=torch.float32,device=device)
        

In [55]:
my_seq=list(range(30))
for x,y in data_iter_random(my_seq,batch_size=2,num_steps=6):
    print("x",x," y",y)

x tensor([[12., 13., 14., 15., 16., 17.],
        [ 0.,  1.,  2.,  3.,  4.,  5.]], device='cuda:0')  y tensor([[13., 14., 15., 16., 17., 18.],
        [ 1.,  2.,  3.,  4.,  5.,  6.]], device='cuda:0')
x tensor([[18., 19., 20., 21., 22., 23.],
        [ 6.,  7.,  8.,  9., 10., 11.]], device='cuda:0')  y tensor([[19., 20., 21., 22., 23., 24.],
        [ 7.,  8.,  9., 10., 11., 12.]], device='cuda:0')


In [57]:
def data_iter_consecutive(corpus_indices,batch_size,num_steps,device=None):
    if device is None:
        device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    corpus_indices=torch.tensor(corpus_indices,dtype=torch.float32,device=device)
    data_len=len(corpus_indices)
    batch_len=data_len//batch_size
    indices=corpus_indices[0:batch_size*batch_len].view(batch_size,batch_len)
    epoch_size=(batch_len-1)//num_steps
    for i in range(epoch_size):
        i=i*num_steps
        x=indices[:,i:i+num_steps]
        y=indices[:,i+1:i+num_steps+1]
        yield x,y

In [59]:
for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')


X:  tensor([[ 0.,  1.,  2.,  3.,  4.,  5.],
        [15., 16., 17., 18., 19., 20.]], device='cuda:0') 
Y: tensor([[ 1.,  2.,  3.,  4.,  5.,  6.],
        [16., 17., 18., 19., 20., 21.]], device='cuda:0') 

X:  tensor([[ 6.,  7.,  8.,  9., 10., 11.],
        [21., 22., 23., 24., 25., 26.]], device='cuda:0') 
Y: tensor([[ 7.,  8.,  9., 10., 11., 12.],
        [22., 23., 24., 25., 26., 27.]], device='cuda:0') 



# 6.4 循环神经网络从零实现

In [62]:
import time
import math 
import numpy as np
import torch
from torch import nn,optim
import torch.nn.functional as F

import sys
sys.path.append("..")
import d2lzh_pytorch as d2l

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()


In [84]:
 torch.set_printoptions(threshold=5000) 

In [88]:
def one_hot(x, n_class, dtype=torch.float32):
    # X shape: (batch), output shape: (batch, n_class)
    x = x.long()
    res = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)
    res.scatter_(1, x.view(-1, 1), 1)
    return res
x = torch.tensor([0, 2])
y=one_hot(x, 10)
y

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])

In [94]:
def to_onehot(x,n_class):
    return[one_hot(x[:,i],n_class) for i in range(x.shape[1])]
x=torch.arange(10).view(2,5)
inputs=to_onehot(x,12)
print(len(inputs),inputs[0].shape)

5 torch.Size([2, 12])


In [95]:
x

tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])

In [96]:
inputs

[tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]]),
 tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]]),
 tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]]),
 tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]]),
 tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])]

In [101]:
num_inputs,num_hiddens,num_outputs=vocab_size,256,vocab_size
print("will use", device)

def get_params():
    def _one(shape):
        ts=torch.tensor(np.random.normal(0,0.01,size=shape),device=device,dtype=torch.float32)
        return torch.nn.Parameter(ts,requires_grad=True)
    # 隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens, device=device, requires_grad=True))
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, requires_grad=True))
    return nn.ParameterList([W_xh, W_hh, b_h, W_hq, b_q])


will use cuda


In [156]:
def init_rnn_state(batch_size,num_hiddens,device):
    return(torch.zeros((batch_size,num_hiddens),device=device),)

In [160]:
def rnn(inputs, state, params):
    # inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)的矩阵
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state#state是tuple,H为state第一个,是tensor
    outputs = []
    for X in inputs:
        H = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(H, W_hh) + b_h)
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)


In [166]:
state = init_rnn_state(X.shape[0], num_hiddens, device)
inputs = to_onehot(X.to(device), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
print(len(outputs), outputs[0].shape, state_new[0].shape)


6 torch.Size([2, 1027]) torch.Size([2, 256])


In [162]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
    num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, device)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        # 将上一时间步的输出作为当前时间步的输入
        X = to_onehot(torch.tensor([[output[-1]]], device=device), vocab_size)
        # 计算输出和更新隐藏状态
        (Y, state) = rnn(X, state, params)
        # 下一个时间步的输入是prefix里的字符或者当前的最佳预测字符
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])


In [163]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
device, idx_to_char, char_to_idx)


'分开夫耿败惚茶正令时斜蜥'

In [192]:
def grad_clipping(params,theta,device):
    norm=torch.tensor([0.0],device=device)
    for param in params:
        norm+=(param.grad.data**2).sum()
    norm=norm.sqrt().item()
    if norm>theta:
        for param in params:
            param.grad.data*=(theta/norm)

In [193]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period,
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random
    else:
        data_iter_fn = d2l.data_iter_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        if not is_random_iter:  # 如使用相邻采样，在epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens, device)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, device)
        for X, Y in data_iter:
            if is_random_iter:  # 如使用随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens, device)
            else:  # 否则需要使用detach函数从计算图分离隐藏状态
                for s in state:
                    s.detach_()
            
            inputs = to_onehot(X, vocab_size)
            # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵
            (outputs, state) = rnn(inputs, state, params)
            # 拼接之后形状为(num_steps * batch_size, vocab_size)
            outputs = torch.cat(outputs, dim=0)
            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # batch * num_steps 的向量，这样跟输出的行一一对应
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            # 使用交叉熵损失计算平均分类误差
            l = loss(outputs, y.long())
            
            # 梯度清0
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params, clipping_theta, device)  # 裁剪梯度
            d2l.sgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, device, idx_to_char, char_to_idx))

In [194]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']


In [195]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
vocab_size, device, corpus_indices, idx_to_char,
char_to_idx, True, num_epochs, num_steps, lr,
clipping_theta, batch_size, pred_period, pred_len,
prefixes)


epoch 50, perplexity 69.644647, time 0.17 sec
 - 分开 我想要这生  爱有你的手写 我想你这你  爱有你有我  爱有你有你的 爱 我有你的可爱 我想要这不
 - 不分开  说你你的你  哼什么我不 我想要再想你 想知你这 我 我有你这 我有你的可爱 我有你这想  爱有
epoch 100, perplexity 10.139438, time 0.16 sec
 - 分开 一直用它三步四步望著天 看星星 一颗两颗三颗四步望著天 看星星 一颗两颗三颗四颗望著天 看星星 一
 - 不分开只 我想能再 你 我知了这 你知我这多功 飞知后觉 快使我有轻功 飞知后双 快使我有轻功  哼穿好 
epoch 150, perplexity 2.919702, time 0.16 sec
 - 分开 有短在人留 我爱往拖抽离 如果我遇见你是一场悲剧 我可以让生命就这样想会 你和还让我的糗  我 你
 - 不分开期 我后能爸 你我已外不想 爸说了我的手你 让我有红了 这场三么 你说完空出 白色蜡烛 温暖了空出 
epoch 200, perplexity 1.590531, time 0.16 sec
 - 分开的凯萨琳 包括安猫斑鸠 腿短毛不多 除天都没有喝水也能活 脑袋瓜有一点秀逗 猎物死了它比谁都难过 印
 - 不分开期 我叫你爸 你打我妈 这样对吗 每这梦中 不敢感动 没有梦 连不知轻重 不有苦衷 问候我回轻的菸 
epoch 250, perplexity 1.288062, time 0.16 sec
 - 分开球 像什不起天 不是都在难喝 为伤透 娘子她人在江南等我 泪不休 语沉默 娘子她人在江南等我 泪不休
 - 不分开期把的胖女巫 用拉丁文念咒语啦啦呜 她养的黑猫笑起来像哭 啦啦啦呜 再来我遇见得猜 看满个人中经刚不


In [196]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
vocab_size, device, corpus_indices, idx_to_char,
char_to_idx, False, num_epochs, num_steps, lr,
clipping_theta, batch_size, pred_period, pred_len,
prefixes)


epoch 50, perplexity 60.713819, time 0.16 sec
 - 分开 我不要的爱写 我哼想这 快使我有 你谁我有 你谁我有 你谁我有 你谁我有 你谁我有 你谁我有 你谁
 - 不分开 我不要的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏
epoch 100, perplexity 7.063266, time 0.15 sec
 - 分开 我想要 你怎么 一九四有 在来一碗热粥 配上几斤的牛肉 我说你没有 我马儿有些瘦 你不么 一直走 
 - 不分开觉 你样经离 我已在这开 我的往实 全暖了空  让没有在 我有多难熬  没有你在我有多烦熬多烦恼  
epoch 150, perplexity 2.158558, time 0.16 sec
 - 分开 我已能 是果有 装一了有信 我唱 他些啊 旧什么 是一些的霜片开 干什么 干什么 东行我有脚踢的 
 - 不分开觉 你已经离 我已不觉 我想没真 你的梦空 有自不同 你你心用 不没有同 你一场痛 不自不同 你你心
epoch 200, perplexity 1.304530, time 0.16 sec
 - 分开 我说啊 是是于睛看手 不有得很隐密 它隔壁一个人 什么都会值得去做 我想大声宣布 对你依依不舍 连
 - 不分开觉 你已经离 我跟不这个过 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生活 不知不觉 
epoch 250, perplexity 1.173610, time 0.16 sec
 - 分开 我的啊 如果载的烟痛 有直人对医 谁让它停留的 它涯拥有 只天黄双个屈 一身正气 他们儿离我习惯 
 - 不分开觉 你已经离 我知不这样的坟墓 古会里一格荒芜 长满杂草的泥土 不会骑扫把的胖女巫 用拉丁文念咒语啦


# 6.5 循环神经网络的简介实现

In [199]:
import time
import math 
import numpy as np
import torch
from torch import nn,optim
import torch.nn.functional as F
import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
device=torch.device(torch.device("cuda"))
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()


In [200]:
num_hiddens=256
rnn_layer=nn.RNN(input_size=vocab_size,hidden_size=num_hiddens)

In [201]:
num_steps=35
batch_size=2
state=None
x=torch.rand(num_steps,batch_size,vocab_size)
y,state_new=rnn_layer(x,state)
print(y.shape,len(state_new),state_new[0].shape)


torch.Size([35, 2, 256]) 1 torch.Size([2, 256])


In [206]:
class rnnmodel(nn.Module):
    def __init__(self,rnn_layer,vocab_size):
        super(rnnmodel,self).__init__()
        self.rnn=rnn_layer
        self.hidden_size=rnn_layer.hidden_size*(2 if rnn_layer.bidirectional else 1)
        self.vocab_size=vocab_size
        self.dense=nn.Linear(self.hidden_size,vocab_size)
        self.state=None
    def forward(self,inputs,state):
        x=d2l.to_onehot(inputs,self.vocab_size)
        y,self.state=self.rnn(torch.stack(x),state)
        output=self.dense(y.view(-1,y.shape[-1]))
        return output, self.state

In [242]:
def predict_rnn_pytorch(prefix,num_chars,model,vocab_size,device,
                       idx_to_char,char_to_idx):
    state=None
    output=[char_to_idx[prefix[0]]]
    for t in range(num_chars+len(prefix)-1):
        x=torch.tensor([output[-1]],device=device).view(1,1)
        if state is not None:
            if isinstance(state,tuple):#判断state是否为tuple
                state=(state[0].to(device),state[1].to(device))
            else:
                state=state.to(device)
            
        (y,state)=model(x,state)
        if t<len(prefix)-1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(y.argmax(dim=1).item()))
    return "".join([idx_to_char[i] for i in output])
            

In [243]:
model = rnnmodel(rnn_layer, vocab_size).to(device)
predict_rnn_pytorch('分开', 10, model, vocab_size, device, idx_to_char, char_to_idx)


'分开梭达两梭两梭两梭两梭'

In [244]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    state = None
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = d2l.data_iter_consecutive(corpus_indices, batch_size, num_steps, device) # 相邻采样
        for X, Y in data_iter:
            if state is not None:
                # 使用detach函数从计算图分离隐藏状态, 这是为了
                # 使模型参数的梯度计算只依赖一次迭代读取的小批量序列(防止梯度计算开销太大)
                if isinstance (state, tuple): # LSTM, state:(h, c)  
                    state = (state[0].detach(), state[1].detach())
                else:   
                    state = state.detach()
    
            (output, state) = model(X, state) # output: 形状为(num_steps * batch_size, vocab_size)
            
            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # batch * num_steps 的向量，这样跟输出的行一一对应
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            l = loss(output, y.long())
            
            optimizer.zero_grad()
            l.backward()
            # 梯度裁剪
            d2l.grad_clipping(model.parameters(), clipping_theta, device)
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        
        try:
            perplexity = math.exp(l_sum / n)
        except OverflowError:
            perplexity = float('inf')
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, perplexity, time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_pytorch(
                    prefix, pred_len, model, vocab_size, device, idx_to_char,
                    char_to_idx))

In [245]:
num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e-3, 1e-2 # 注意这里的学习率设置
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
corpus_indices, idx_to_char, char_to_idx,
num_epochs, num_steps, lr, clipping_theta,
batch_size, pred_period, pred_len, prefixes)


epoch 50, perplexity 8.944099, time 0.05 sec
 - 分开 我不了 爱 我不能再想 我不 我不 我想 你不要再想  想要你不能再想 我想要你不能我想 你想你我
 - 不分开 我想你你爱你我不  想要你不能再想 我想你你不 我不 再想要我不要再想 我想 你不能我想 你想你我
epoch 100, perplexity 1.246496, time 0.05 sec
 - 分开 我的始爸着 我想的 不能再想你这样打我妈  爱你 你 我不懂 你怎么我 你 你知了口 是可以可以简
 - 不分开始 会生活  不能不觉 我不能看想  说你说你 手 我痛  为什么 不要再一个 不想就这样牵着你的手
epoch 150, perplexity 1.064673, time 0.05 sec
 - 分开 我的始爸你 会想要再想要我想 你说 分不 我想要 说你说你 你怎么 想要再这样打我妈妈 我说的话你
 - 不分开始 像生活 我爱上的可爱 悲笑 我有人 对你 回忆的 得它上话 一场悲剧 你的完美主义 太彻底 让我
epoch 200, perplexity 1.032861, time 0.05 sec
 - 分开 我的泪看着 就想 你怎么我的妈都会 再想要 你是我不想你  没说你烦在 我 懂不要再这样 甜蜜回让
 - 不分开不 我不要再想你 爱情来的太快就像龙卷风 离不开暴风圈来不及逃 我不能再想 我不能再想 我不 我不 
epoch 250, perplexity 1.031139, time 0.05 sec
 - 分开 我的泪看着 就想 我想大听不知后觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 
 - 不分开  没担了 还对我了大堡  说散 你想很久了吧? 我的认真败给黑色幽默 走过了很多地方 我来到伊斯坦


# 6.6 通过时间反向传播

# 6.7 门控循环单元

$$H_t=Z_t \odot H_{t-1}+(1-Z_t)\odot \tilde{H_t}$$

其中$H_t$是时间步的隐藏状态,$Z_t$是更新门的结果,$\tilde{H_t}$是候选隐藏状态,其计算过程为:

$$\tilde{H_t}=\tanh (x_t W_{xh}+(R_t \odot H_{t-1})W_{hh}+b_h)$$

其中$w$为不同的权重参数,$b$为不同的偏差函数,$R_t$ 为重置门输出结果,重置门与更新门的计算如下:

$$R_t=\sigma(x_tw_{xr}+H_{t-1}W_{hr}+b_r) \\Z_t=\sigma(x_tw_{xz}+H_{t-1}W_{hz}+b_r)$$

最后可以的出$y$的计算如下:
$$y=H_tW_{hq}+b_q$$

In [246]:
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()


In [247]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)


will use cuda


In [253]:
def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))
    
    W_xz, W_hz, b_z = _three()  # 更新门参数
    W_xr, W_hr, b_r = _three()  # 重置门参数
    W_xh, W_hh, b_h = _three()  # 候选隐藏状态参数
    
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q])

In [254]:
def init_gru_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )


In [255]:
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        Z = torch.sigmoid(torch.matmul(X, W_xz) + torch.matmul(H, W_hz) + b_z)
        R = torch.sigmoid(torch.matmul(X, W_xr) + torch.matmul(H, W_hr) + b_r)
        H_tilda = torch.tanh(torch.matmul(X, W_xh) + R * torch.matmul(H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

In [256]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']


In [257]:
d2l.train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens,
vocab_size, device, corpus_indices, idx_to_char,
char_to_idx, False, num_epochs, num_steps, lr,
clipping_theta, batch_size, pred_period, pred_len,
prefixes)


epoch 40, perplexity 149.598042, time 0.40 sec
 - 分开 我想你 我想你 我想你 你不不 我想你 你不不 我想你 你不不 我想你 你不不 我想你 你不不 我
 - 不分开 我想你 我想你 我想你 你不不 我想你 你不不 我想你 你不不 我想你 你不不 我想你 你不不 我
epoch 80, perplexity 32.808943, time 0.39 sec
 - 分开 我想要你想想你 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想
 - 不分开 我不要再想你 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 
epoch 120, perplexity 5.030714, time 0.39 sec
 - 分开 一直个人落人的响尾 古著一双吴昏 你的完美主义 太彻底 它手的话像语听的暴            
 - 不分开 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生活
epoch 160, perplexity 1.438310, time 0.40 sec
 - 分开 一个个人落边 它在许愿 习人了空屋 白色蜡烛 温暖了空屋 白色蜡烛 温暖了空屋 白色蜡烛 温暖了空
 - 不分开 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生活


In [258]:
lr=1e-2
gru_layer=nn.GRU(input_size=vocab_size,hidden_size=num_hiddens)
model=d2l.RNNModel(gru_layer,vocab_size).to(device)
d2l.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
corpus_indices, idx_to_char, char_to_idx,
num_epochs, num_steps, lr, clipping_theta,
batch_size, pred_period, pred_len, prefixes)


epoch 40, perplexity 1.016776, time 0.05 sec
 - 分开始共渡每一天 手牵手 一步两步三步四步望著天 看星星 一颗两颗三颗四颗 连成线背著背默默许下心愿 看
 - 不分开任督二脉 干什么 干什么 东亚病夫的招牌 干什么 干什么 已被我一脚踢开 快使用双截棍 哼哼哈兮 快
epoch 80, perplexity 1.018032, time 0.05 sec
 - 分开始共渡每一天 手牵手 一步两步三步四步望著天 看星星 一颗两颗三颗四颗 连成线背著背默默许下心愿 看
 - 不分开暴风圈来不及逃 我不能再想 我不能再想 我不 我不 我不能 爱情走的太快就像龙卷风 不能承受我已无处
epoch 120, perplexity 1.011281, time 0.05 sec
 - 分开 誓言太沉重泪被纵容 脸上汹涌失控 城市霓虹 不安跳动 染红夜空 过去种种 象一场梦 不敢去碰 一想
 - 不分开暴风圈来不及逃 我不能再想 我不能再想 我不 我不 我不能 爱情走的太快就像龙卷风 不能承受我已无处
epoch 160, perplexity 1.011430, time 0.05 sec
 - 分开 学少林跟武当 快使用双截棍 哼哼哈兮 快使用双截棍柔中带刚 想要去河南嵩山 学少林跟武当 快使用双
 - 不分开始打我 你知道 安静的听你撒娇 看你睡著一直到老 就是开不了口让她知道 就是那么简单几句 我办不到 


# 长短期记忆 LSTM

最后输出$y$ 的计算如下:(下述中$W$与$b$分别为对应的系数)
$$y=H_tW_{hq}+b_q$$
其中$H$ 为隐藏状态,计算如下:
$$H_t=O_t \odot \tanh (C_t)$$

其中$O_t$ 和$C_t$ 分别为当前输出门的结果和当前记忆细胞的结果,计算如下:

$$O_t=\sigma (x_tW_{x0}+H_{t-1}W_{h0}+b_0)
\\C_t=F_t \odot C_{t-1}+I_t \odot \tilde{C_t}$$

其中 $F_t$,$I_t$,$\tilde{C_t}$分别为遗忘门的结果,输入门的结果和候选记忆细胞,其计算如下:

$$
F_t=\sigma (x_tW_{xi}+H_{t-1}W_{hi}+b_i)\\
I_t=\sigma (x_tW_{xf}+H_{t-1}W_{hf}+b_f)\\
\tilde{C_t}=\tanh (x_tW_{xc}+H_{t-1}W_{hc}+b_c)
$$

In [265]:
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

print(torch.__version__, device)

1.2.0 cuda


In [266]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))
    
    W_xi, W_hi, b_i = _three()  # 输入门参数
    W_xf, W_hf, b_f = _three()  # 遗忘门参数
    W_xo, W_ho, b_o = _three()  # 输出门参数
    W_xc, W_hc, b_c = _three()  # 候选记忆细胞参数
    
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q])

will use cuda


In [267]:
def init_lstm_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), 
            torch.zeros((batch_size, num_hiddens), device=device))

In [268]:
def lstm(inputs, state, params):
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q] = params
    (H, C) = state
    outputs = []
    for X in inputs:
        I = torch.sigmoid(torch.matmul(X, W_xi) + torch.matmul(H, W_hi) + b_i)
        F = torch.sigmoid(torch.matmul(X, W_xf) + torch.matmul(H, W_hf) + b_f)
        O = torch.sigmoid(torch.matmul(X, W_xo) + torch.matmul(H, W_ho) + b_o)
        C_tilda = torch.tanh(torch.matmul(X, W_xc) + torch.matmul(H, W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * C.tanh()
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, C)

In [269]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [270]:
d2l.train_and_predict_rnn(lstm, get_params, init_lstm_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)

epoch 40, perplexity 214.763703, time 0.44 sec
 - 分开 我不不 我不我你 我不你我 我不你我 我不你我 我不你我 我不你我 我不你我 我不你我 我不你我 
 - 不分开 我不不我 我不你我 我不你我 我不你我 我不你我 我不你我 我不你我 我不你我 我不你我 我不你我
epoch 80, perplexity 66.612539, time 0.44 sec
 - 分开 我想你你的你 我想要你 我不要这 我不要这 我不要这 我不要这 我不要这 我不要这 我不要这 我不
 - 不分开 我想你你的你 我不要你 我不要这 我不要这 我不要这 我不要这 我不要这 我不要这 我不要这 我不
epoch 120, perplexity 15.502695, time 0.44 sec
 - 分开 我想你你 我有多烦 我不要再生活 你知去觉 我该了这生活 我知好觉 我不了这生活 后知后觉 我该好
 - 不分开 我不要你 我不要这样 我不好觉 我不不好生活 后知后觉 我该好好生活 我知好觉 我不了这生活 后知
epoch 160, perplexity 3.939239, time 0.45 sec
 - 分开 你是你的话有 一悔  又念很 我想就这些信着你 干什么手手得多 他物棍跟了嵩山 我说你的话快有样 
 - 不分开 我想要你 我我要多熬 我不你没生活 你知你觉一直 连直歌邻居都到到我现想的感受 河血的美蜜 吹吹着


In [271]:
lr=1e-2
lstm_layer=nn.LSTM(input_size=vocab_size,hidden_size=num_hiddens)
model=d2l.RNNModel(lstm_layer,vocab_size)
d2l.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
corpus_indices, idx_to_char, char_to_idx,
num_epochs, num_steps, lr, clipping_theta,
batch_size, pred_period, pred_len, prefixes)


epoch 40, perplexity 1.024684, time 0.05 sec
 - 分开始移动 回到当初爱你的时空 停格内容不忠 所有回忆对着我进攻       古巴比伦王颁布了汉摩拉比法
 - 不分开始移动 回到当初爱你的时空 停格内容不忠 所有回忆对着我进攻       古巴比伦王颁布了汉摩拉比法
epoch 80, perplexity 1.013150, time 0.05 sec
 - 分开始移动 回到当初爱你的时空 停格内容不忠 所有回忆对着我进攻 我的伤口被你拆封 誓言太沉重泪被纵容 
 - 不分开  能不能够永远单纯没有悲哀 我 想带你骑单车 我 想和你看棒球 想这样没担忧 唱着歌 一直走 我想
epoch 120, perplexity 1.012132, time 0.05 sec
 - 分开始移动 回到当初爱你的时空 停格内容不忠 所有回忆对着我进攻 我的伤口被你拆封 誓言太沉重泪被纵容 
 - 不分开 爱能不能够永远单纯没有悲哀 我 想带你骑单车 我 想和你看棒球 想这样没担忧 唱着歌 一直走 我想
epoch 160, perplexity 1.008434, time 0.05 sec
 - 分开始移动 回到当初爱你的时空 停格内容不忠 所有回忆对着我进攻 我的伤口被你拆封 誓言太沉重泪被纵容 
 - 不分开 爱能不能够永远单纯没有悲哀 我 想带你骑单车 我 想和你看棒球 想这样没担忧 唱着歌 一直走 我想
